From VENUES_DIR to EXTRACTED_VENUES

In [1]:

VENUES_DIR = '../data/output/4square/'
import glob
import csv
import json
from ast import literal_eval

In [2]:
import json

In [10]:
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display
TARGET_KEYS = ['id', 'name', 'location', 'categories', 'verified', 'stats']
all_venues = []

for fn in glob.glob(VENUES_DIR+'*'):
    print fn
    with open(fn) as f:
        for line in f:
            swne, vcnt, data = line.split('\t')
            data = literal_eval(data)
            venues = data['venues']
            all_venues.extend(venues)

len(all_venues)

../data/output/4square\10000.txt
../data/output/4square\12000.txt
../data/output/4square\14000.txt
../data/output/4square\16000.txt
../data/output/4square\18000.txt
../data/output/4square\2000.txt
../data/output/4square\20000.txt
../data/output/4square\21481.txt
../data/output/4square\4000.txt
../data/output/4square\6000.txt
../data/output/4square\8000.txt


43004

In [27]:
df_venues = json_normalize(all_venues)

In [54]:
import shapely.geometry as shpgeo

In [55]:
df_venues['categories.name'] = df_venues.categories.apply(lambda x: x[0]['name'] if x else '')
df_venues['categories.id'] = df_venues.categories.apply(lambda x: x[0]['id'] if x else '')
df_venues['geometry'] = df_venues.apply(lambda x: shpgeo.Point(x['location.lng'], x['location.lat']), axis=1)

In [60]:
df_venues['location.isFuzzed'].value_counts()

True    2
dtype: int64

In [61]:
print df_venues['location.labeledLatLngs'].apply(lambda x: x[0]['label'] if type(x)!=float else 0).value_counts()
df_venues.apply(lambda x: (x['location.labeledLatLngs'][0]['lng']/x['location.lng'] )
                            if type(x['location.labeledLatLngs'])!=float else 0, axis=1).value_counts()

display    42793
0            211
dtype: int64


1    42793
0      211
dtype: int64

In [68]:
columns_ = ['id', 'geometry', 'name', 'stats.checkinsCount', 'stats.tipCount', 'stats.usersCount','categories.name']
df_no_dup = df_venues[columns].drop_duplicates('id').copy()

In [72]:

df_no_dup.columns = ['id', 'geometry', 'name', 'checkins', 'tips', 'users','category']


In [76]:
import geopandas as gp
with open(EXTRACTED_VENUES, 'w') as f:
    f.write(gp.GeoDataFrame(df_no_dup).to_json())